In [1]:
import os
from pprint import pprint
import os.path as osp
from datetime import datetime

import numpy as np
import pandas as pd

import baposgmcp.plot as plot_utils
from baposgmcp.run import compile_result_files

results_dir = "/home/jonathon/baposgmcp_results/Driving14x14WideRoundAbout-n2-v0/time_exp"
# output file
time_str = datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
output_filename = f"aggregated_results_{time_str}.csv"
output_filepath = osp.join(results_dir, output_filename)

In [2]:
# input files
result_filepaths = []
for dirpath, dirnames, filenames in os.walk(results_dir):
    for fname in filenames:
        if fname.endswith("episodes.csv"):
            result_filepaths.append(os.path.join(dirpath, fname))

# Import data
ep_df = compile_result_files(result_filepaths, verbose=True, n_procs=4)

Loading and concatting 168 files
Compiling using 4
[pid=965664] 4/42 processed[pid=965667] 4/42 processed[pid=965665] 4/42 processed


[pid=965666] 4/42 processed
[pid=965667] 8/42 processed[pid=965664] 8/42 processed

[pid=965665] 8/42 processed
[pid=965666] 8/42 processed
[pid=965667] 12/42 processed
[pid=965664] 12/42 processed
[pid=965665] 12/42 processed
[pid=965666] 12/42 processed
[pid=965667] 16/42 processed
[pid=965665] 16/42 processed
[pid=965664] 16/42 processed
[pid=965666] 16/42 processed
[pid=965667] 20/42 processed
[pid=965665] 20/42 processed
[pid=965666] 20/42 processed[pid=965664] 20/42 processed

[pid=965667] 24/42 processed
[pid=965665] 24/42 processed
[pid=965666] 24/42 processed
[pid=965667] 28/42 processed
[pid=965664] 24/42 processed
[pid=965665] 28/42 processed
[pid=965666] 28/42 processed
[pid=965664] 28/42 processed
[pid=965667] 32/42 processed
[pid=965666] 32/42 processed
[pid=965665] 32/42 processed
[pid=965666] 36/42 processed
[pid=965664] 32/42 processed


In [3]:
# Outcomes parse functions
def parse_win(row):
    return int(row["episode_outcome"] == 'WIN')

def parse_loss(row):
    return int(row["episode_outcome"] == 'LOSS')

def parse_draw(row):
    return int(row["episode_outcome"] == 'DRAW')

def parse_na(row):
    return int(row["episode_outcome"] not in ('WIN', 'LOSS', 'DRAW'))

In [5]:
constants = [
    "agent_id",
    "env_id",
    "time_limit",
    "episode_step_limit",
    "discount",
    "c",
    "c_init",
    "c_base",
    "truncated",
    "action_selection",
    "dirichlet_alpha",
    "root_exploration_fraction",
    "reinvigorator",
    "known_bounds",
    "extra_particles_prop",
    "step_limit",
    "epsilon",
    "num_sims",
    "search_time_limit",
    "num_episodes",
    "policy_prior_map",
    "fixed_policy_id",
    "meta_policy_dict",
]
replaced = [
    # replaced by number of episodes completed
    "num_episodes",
    # removed/superseded by above
    "episode_number",
    # parsed into num_outcome_...
    "episode_outcome",
    # removed/superseded by 'episode_outcome'
    "episode_done",
]
# take first value in grouped df
first_keys = [
    "exp_id",
    "exp_seed",
]

# values that will be summed across groups
outcome_col_names = ["WIN", "LOSS", "DRAW", "NA"]
sum_keys = outcome_col_names

mean_keys = [
    'search_time',
    'update_time',
    'reinvigoration_time',
    'evaluation_time',
    'policy_calls',
    'inference_time',
    'search_depth',
    'min_value',
    'max_value',
    'episode_return',
    'episode_discounted_return',
    'episode_steps',
    'episode_time'
]

In [6]:
# Keys which DF is grouped by
group_keys = [
    "policy_id",
    # "coplayer_policy_id"    # added based on num agents in the env
]

# Adding coplayer policy id column
if len(ep_df["agent_id"].unique().tolist()) == 2:
    ep_df = plot_utils.add_df_coplayer_policy_id(ep_df)
    group_keys.append("coplayer_policy_id")
else:
    ep_df = plot_utils.add_df_multiple_coplayer_policy_id(ep_df)
    for c in ep_df.columns:
        if c.startswith("coplayer_policy_id"):
            group_keys.append(c)
    group_keys.append("agent_id")
    constants.remove("agent_id")
    
# replace num_episodes with actual number of episodes completed
ep_df["num_episodes"] = ep_df.groupby(group_keys)["num_episodes"].transform(len)

# parse episode outcomes into seperate columns
outcome_col_names = ["WIN", "LOSS", "DRAW", "NA"]
for k, fn in zip(outcome_col_names, [parse_win, parse_loss, parse_draw, parse_na]):
    ep_df[k] = ep_df.apply(fn, axis=1)
    
# 

assigned_keys = set(group_keys + constants + replaced + first_keys + sum_keys + mean_keys)
belief_stat_keys = [c for c in ep_df if c not in assigned_keys]

print("\nExperiments Seeds:", ep_df["exp_seed"].unique())
    
print("\nColumns")
print("-------")
for c in ep_df.columns:
    print(f'    "{c}",')
    


Experiments Seeds: [1 0]

Columns
-------
    "exp_id",
    "agent_id",
    "env_id",
    "exp_seed",
    "num_episodes",
    "time_limit",
    "episode_step_limit",
    "policy_id",
    "discount",
    "c",
    "action_selection",
    "dirichlet_alpha",
    "root_exploration_fraction",
    "reinvigorator",
    "known_bounds",
    "extra_particles_prop",
    "step_limit",
    "epsilon",
    "num_sims",
    "policy_prior_map",
    "meta_policy_dict",
    "search_time_limit",
    "truncated",
    "action_dist_distance_0_mean",
    "action_dist_distance_0_std",
    "action_dist_distance_1_mean",
    "action_dist_distance_1_std",
    "action_dist_distance_0_0",
    "action_dist_distance_0_1",
    "action_dist_distance_0_2",
    "action_dist_distance_0_3",
    "action_dist_distance_0_4",
    "action_dist_distance_0_5",
    "action_dist_distance_0_6",
    "action_dist_distance_0_7",
    "action_dist_distance_0_8",
    "action_dist_distance_0_9",
    "action_dist_distance_0_10",
    "action_

In [7]:
# group by and then aggregate
gb = ep_df.groupby(group_keys)
columns = set(list(ep_df.columns))

agg_dict = {}
for k in first_keys:
    if k in columns:
        agg_dict[k] = pd.NamedAgg(column=k, aggfunc="min")
    else:
        print(f"Column {k} missing")

for k in constants:
    if k in columns:
        agg_dict[k] = pd.NamedAgg(column=k, aggfunc="first")
    else:
        print(f"Column {k} missing")

for k in sum_keys:
    if k in columns:
        agg_dict[f"num_{k}"] = pd.NamedAgg(column=k, aggfunc="sum")
    else:
        print(f"Column {k} missing")

for k in mean_keys:
    if k in columns:
        agg_dict[f"{k}_mean"] = pd.NamedAgg(column=k, aggfunc="mean")
        agg_dict[f"{k}_std"] = pd.NamedAgg(column=k, aggfunc="std")
        agg_dict[f"{k}_min"] = pd.NamedAgg(column=k, aggfunc="min")
        agg_dict[f"{k}_max"] = pd.NamedAgg(column=k, aggfunc="max")
    else:
        print(f"Column {k} missing")

for k in belief_stat_keys:
    if k not in columns:
        print(f"Column {k} missing")
    elif k.endswith("_mean") or k.endswith("_std"):
        agg_dict[f"{k}"] = pd.NamedAgg(column=k, aggfunc="mean")
    else:
        agg_dict[f"{k}_mean"] = pd.NamedAgg(column=k, aggfunc="mean")
        agg_dict[f"{k}_std"] = pd.NamedAgg(column=k, aggfunc="std")
        # get count of non nan values since this varies for belief stats
        # based on step number
        agg_dict[f"{k}_n"] = pd.NamedAgg(column=k, aggfunc="count")

    
gb_agg = gb.agg(**agg_dict)
gb_agg

Column c_init missing
Column c_base missing


exp_id  \
policy_id                                          coplayer_policy_id                                           
Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0   baposgmcp-random_searchtimelimit0.1_truncatedFalse      20   
                                                   baposgmcp-random_searchtimelimit10_truncatedFalse       32   
                                                   baposgmcp-random_searchtimelimit1_truncatedFalse        24   
                                                   baposgmcp-random_searchtimelimit20_truncatedFalse       36   
                                                   baposgmcp-random_searchtimelimit5_truncatedFalse        28   
...                                                                                                       ...   
ucbmcp_metasoftmax_searchtimelimit20_truncatedTrue Driving14x14WideRoundAbout-n2-v0/klr_k3_seed0-v0        59   
ucbmcp_metasoftmax_searchtimelimit5_truncatedTrue  Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0        48   
                                                   Driving14x14WideRoundAbout-n2-v0/klr_k1_seed0-v0        49   
                                                   Driving14x14WideRoundAbout-n2-v0/klr_k2_seed0-v0        50   
                                                   Driving14x14WideRoundAbout-n2-v0/klr_k3_seed0-v0        51   

                                                                                                       exp_seed  \
policy_id                                          coplayer_policy_id                                             
Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0   baposgmcp-random_searchtimelimit0.1_truncatedFalse         0   
                                                   baposgmcp-random_searchtimelimit10_truncatedFalse          0   
                                                   baposgmcp-random_searchtimelimit1_truncatedFalse           0   
                                                   baposgmcp-random_searchtimelimit20_truncatedFalse          0   
                                                   baposgmcp-random_searchtimelimit5_truncatedFalse           0   
...                                                                                                         ...   
ucbmcp_metasoftmax_searchtimelimit20_truncatedTrue Driving14x14WideRoundAbout-n2-v0/klr_k3_seed0-v0           0   
ucbmcp_metasoftmax_searchtimelimit5_truncatedTrue  Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0           0   
                                                   Driving14x14WideRoundAbout-n2-v0/klr_k1_seed0-v0           0   
                                                   Driving14x14WideRoundAbout-n2-v0/klr_k2_seed0-v0           0   
                                                   Driving14x14WideRoundAbout-n2-v0/klr_k3_seed0-v0           0   

                                                                                                       agent_id  \
policy_id                                          coplayer_policy_id                                             
Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0   baposgmcp-random_searchtimelimit0.1_truncatedFalse         1   
                                                   baposgmcp-random_searchtimelimit10_truncatedFalse          1   
                                                   baposgmcp-random_searchtimelimit1_truncatedFalse           1   
                                                   baposgmcp-random_searchtimelimit20_truncatedFalse          1   
                                                   baposgmcp-random_searchtimelimit5_truncatedFalse           1   
...                                                                                                         ...   
ucbmcp_metasoftmax_searchtimelimit20_truncatedTrue Driving14x14WideRoundAbout-n2-v0/klr_k3_seed0-v0           0   
ucbmcp_metasoftmax_searchtimelimit5_truncatedTrue  Driving14x14WideRoundAbout-n2-v0/klr_k0_seed0-v0   

In [8]:
compiled_df = gb_agg.reset_index()
compiled_df.to_csv(output_filepath)